In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Index
1. Dataset description
2. Exploratory data analysis (EDA)
3. Data Preprocessing and Normailzation
4. Feature Engineering
5. Model Building
6. ROC Curve
7. conclusion

## Dataset description

1. Age : Age of the patient
2. Sex : Sex of the patient
3. exang: exercise induced angina (1 = yes; 0 = no)
4. ca: number of major vessels (0-3)
5. cp : Chest Pain type chest pain type
        Value 1: typical angina
        Value 2: atypical angina
        Value 3: non-anginal pain
        Value 4: asymptomatic
6. trtbps : resting blood pressure (in mm Hg)
7. chol : cholestoral in mg/dl fetched via BMI sensor
8. fbs : (fasting blood sugar > 120 mg/dl) (1 = true; 0 = false)
9. rest_ecg : resting electrocardiographic results
        Value 0: normal
        Value 1: having ST-T wave abnormality (T wave inversions and/or ST elevation or depression of > 0.05 mV)
        Value 2: showing probable or definite left ventricular hypertrophy by Estes' criteria
10. thalach : maximum heart rate achieved
11. target : 0= less chance of heart attack 1= more chance of heart attack

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.metrics import confusion_matrix ,classification_report,precision_score, recall_score ,f1_score 

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression  
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC 
import warnings
warnings.filterwarnings('ignore')

In [ ]:
data = pd.read_csv('/kaggle/input/heart-attack-analysis-prediction-dataset/heart.csv')

In [ ]:
data.head()

## Exploratory data analysis (EDA)
### 1) For numeric data 

* Made histograms to understand distributions 
* Corrplot 
* Pivot table comparing survival rate across numeric variables 

### 2) For Categorical Data 
* Made bar charts to understand balance of classes 
* Made pivot tables to understand relationship with survival 

In [ ]:
data.info()

In [ ]:
data.describe()

In [ ]:
data.describe().columns

In [ ]:
# numeric and categorical value separately
data_num = data[['age','trtbps','chol','thalachh','oldpeak']]
data_cat =data[['sex','cp','fbs','restecg','exng']]

In [ ]:
for i in data_num.columns:
    plt.hist(data_num[i])
    plt.title(i)
    plt.show()

In [ ]:
print(data_num.corr())
sns.heatmap(data_num.corr())

In [ ]:
pd.pivot_table(data, index='output', values=['age','trtbps','chol','thalachh','oldpeak'])

In [ ]:
for i in data_cat.columns:
    sns.barplot(data_cat[i].value_counts().index,data_cat[i].value_counts()).set_title(i)
    plt.show()

In [ ]:
print(pd.pivot_table(data,index='output',columns='sex', values='age'))
print("="*100)
print(pd.pivot_table(data,index='output',columns='cp', values='age'))
print("="*100)
print(pd.pivot_table(data,index='output',columns='fbs', values='age'))
print("="*100)
print(pd.pivot_table(data,index='output',columns='restecg', values='age'))
print("="*100)
print(pd.pivot_table(data,index='output',columns='exng', values='age'))

In [ ]:
print(pd.pivot_table(data,index='output',columns='sex', values='chol'))
print("="*100)
print(pd.pivot_table(data,index='output',columns='cp', values='chol'))
print("="*100)
print(pd.pivot_table(data,index='output',columns='fbs', values='chol'))
print("="*100)
print(pd.pivot_table(data,index='output',columns='restecg', values='chol'))
print("="*100)
print(pd.pivot_table(data,index='output',columns='exng', values='chol'))

## Data Preprocessing and Normailzation
Use the percentile technique to normalize the columns.

In [ ]:
for i in data_num.columns:
    sns.boxplot(data_num[i])
    plt.title(i)
    plt.show()

In [ ]:
def outlinefree(dataCol):
    # sorting column
    sorted(dataCol)
    
    # getting percentile 25 and 27 that will help us for getting IQR (interquartile range)
    Q1,Q3 = np.percentile(dataCol,[25,75])
    
    # getting IQR (interquartile range)
    IQR = Q3-Q1
    
    # getting Lower range error
    LowerRange = Q1-(1.5 * IQR)
    
    # getting upper range error
    UpperRange = Q3+(1.5 * IQR)
    
    # return Lower range and upper range.
    return LowerRange,UpperRange

In [ ]:
lwtrtbps,uptrtbps = outlinefree(data['trtbps'])
lwchol,upchol = outlinefree(data['chol'])
lwoldpeak,upoldpeak = outlinefree(data['oldpeak'])

In [ ]:
data['trtbps'].replace(list(data[data['trtbps'] > uptrtbps].trtbps) ,uptrtbps,inplace=True)
data['chol'].replace(list(data[data['chol'] > upchol].chol) ,upchol,inplace=True)
data['oldpeak'].replace(list(data[data['oldpeak'] > upoldpeak].oldpeak) ,upoldpeak,inplace=True)

## Feature Engineering

In [ ]:
features = data.iloc[:,:-1].values
label = data.iloc[:,-1].values

# Model Building
I like to see how various different models perform with default parameters on dataset and checking F1 score.

1. **LogisticRegression  0.9268292682926829**
2. K-Nearest Neighbor(KNN) 0.7865168539325843
3. support vector classification 0.7999999999999999
4. naive bayes 0.9090909090909092

In [ ]:
#------------------------LogisticRegression-----------------------
X_train, X_test, y_train, y_test= train_test_split(features,label, test_size= 0.25, random_state=102)

classimodel= LogisticRegression()  
classimodel.fit(X_train, y_train)
trainscore =  classimodel.score(X_train,y_train)
testscore =  classimodel.score(X_test,y_test)  

print("test score: {} train score: {}".format(testscore,trainscore),'\n')

y_pred =  classimodel.predict(X_test)

#from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

In [ ]:
print(' f1 score: ',f1_score(y_test, y_pred),'\n')
print(' precision score: ',precision_score(y_test, y_pred),'\n')
print(' recall score: ',recall_score(y_test, y_pred),'\n')
print(classification_report(y_test, y_pred))

In [ ]:
#--------------------------------------K-Nearest Neighbor(KNN)-----------------
X_train, X_test, y_train, y_test= train_test_split(features,label, test_size= 0.25, random_state=193) 


classifier= KNeighborsClassifier()  
knnmodel =  classifier.fit(X_train, y_train) 

trainscore =  knnmodel.score(X_train,y_train)
testscore =  knnmodel.score(X_test,y_test)  

print("test score: {} train score: {}".format(testscore,trainscore),'\n')

y_predknn =  knnmodel.predict(X_test)

print(confusion_matrix(y_test, y_predknn))

In [ ]:
print("f1_score: ",f1_score(y_test, y_predknn),'\n')
print("precision_score: ",precision_score(y_test, y_predknn),'\n')
print("recall_score: ",recall_score(y_test, y_predknn),'\n')
print(classification_report(y_test, y_predknn))


In [ ]:
#------------------------------naive bayes---------------------------
X_train, X_test, y_train, y_test= train_test_split(features,label, test_size= 0.25, random_state=34) 

NBmodel = GaussianNB()  
NBmodel.fit(X_train, y_train) 

trainscore =  NBmodel.score(X_train,y_train)
testscore =  NBmodel.score(X_test,y_test)  

print("test score: {} train score: {}".format(testscore,trainscore),'\n')
y_predNB =  NBmodel.predict(X_test)
print(confusion_matrix(y_test, y_predNB))

In [ ]:
print("f1_score: ",f1_score(y_test, y_predNB),'\n')
print("precision_score: ",precision_score(y_test, y_predNB),'\n')
print("recall_score: ",recall_score(y_test, y_predNB),'\n')
print(classification_report(y_test, y_predNB))

In [ ]:
#-------------------------------- support vector classification -------------------------------------  
X_train, X_test, y_train, y_test= train_test_split(features,label, test_size= 0.25, random_state=8) 

svcmodel = SVC(probability=True)  
svcmodel.fit(X_train, y_train) 

trainscore =  svcmodel.score(X_train,y_train)
testscore =  svcmodel.score(X_test,y_test)  

print("test score: {} train score: {}".format(testscore,trainscore),'\n')

y_predsvc =  svcmodel.predict(X_test)

print(confusion_matrix(y_test, y_predsvc))

In [ ]:
print("f1_score: ",f1_score(y_test, y_predsvc),'\n')
print("precision_score: ",precision_score(y_test, y_predsvc),'\n')
print("recall_score: ",recall_score(y_test, y_predsvc),'\n')
print(classification_report(y_test, y_predsvc),'\n')

## ROC Curve
I like to see how the ROC Curve performs on various different models to get the best Model.

1. **LogisticRegression 0.9214756258234519**
2. K-Nearest Neighbor(KNN) 0.8049407114624506
3. support vector classification 0.7574879227053138
4. naive bayes 0.9104084321475626

In [ ]:
#-------------------------------------- LogisticRegression -------------------------------------
probabilityValues = classimodel.predict_proba(features)[:,1]
#Calculate AUC
auc = roc_auc_score(label,probabilityValues)
print(auc)
#Calculate roc_curve
fpr,tpr, threshold =  roc_curve(label,probabilityValues)
plt.plot([0,1],[0,1], linestyle = '--')
plt.plot(fpr,tpr)

In [ ]:
#-------------------------------------- KNeighborsClassifier -------------------------------------
probabilityValues = knnmodel.predict_proba(features)[:,1]
#Calculate AUC
auc = roc_auc_score(label,probabilityValues)
print(auc)
#Calculate roc_curve
fpr,tpr, threshold =  roc_curve(label,probabilityValues)
plt.plot([0,1],[0,1], linestyle = '--')
plt.plot(fpr,tpr)

In [ ]:
#-------------------------------------- naive bayes -------------------------------------
probabilityValues = NBmodel.predict_proba(features)[:,1]
#Calculate AUC
auc = roc_auc_score(label,probabilityValues)
print(auc)
#Calculate roc_curve
fpr,tpr, threshold =  roc_curve(label,probabilityValues)
plt.plot([0,1],[0,1], linestyle = '--')
plt.plot(fpr,tpr)

In [ ]:
#-------------------------------------- SVC -------------------------------------
probabilityValues = svcmodel.predict_proba(features)[:,1]
#Calculate AUC
auc = roc_auc_score(label,probabilityValues)
print(auc)
#Calculate roc_curve
fpr,tpr, threshold =  roc_curve(label,probabilityValues)
plt.plot([0,1],[0,1], linestyle = '--')
plt.plot(fpr,tpr)

## Conclusion
according to the model and ROC curve, we found the best model for the dataset. **LogisticRegression algorithm** giving us the best result.


    f1 score:  0.9268292682926829
    ROC curve: 0.9214756258234519